In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.ml.feature import StopWordsRemover, Tokenizer

from analysis import WordList
from pyspark.sql.types import IntegerType

In [11]:
spark = SparkSession \
        .builder \
        .master('spark://10.10.28.172:7077') \
        .appName('sentiment_wordlist') \
        .enableHiveSupport() \
        .config("spark.pyfiles", "analysis.py") \
        .getOrCreate()

# Preventing errors of having unused settings in hive-site.xml
spark.sparkContext.setLogLevel('OFF')
# Name of dependency files
spark.sparkContext.addPyFile("analysis.py")

spark.sql('show databases').show()

+------------+
|   namespace|
+------------+
|     default|
|twitter_data|
+------------+



In [14]:
spark.sql('use twitter_data')
query = '''
    SELECT count(*) 
    FROM raw_data
    WHERE text IS NOT NULL
    AND created_at IS NOT NULL
'''

raw_data = spark.sql(query)
raw_data.show(5)

+----+----+-------------------+---------+----------+
|text| geo|                 id|author_id|created_at|
+----+----+-------------------+---------+----------+
|null|null|               null|     null|      null|
|null|null|               null|     null|      null|
|null|null|1496409502500937729|     null|      null|
|null|null|               null|     null|      null|
|null|null|               null|     null|      null|
+----+----+-------------------+---------+----------+
only showing top 5 rows



In [21]:
spark.sql('select count(*) from raw_data where created_at IS NULL OR text IS NULL').show()

+--------+
|count(1)|
+--------+
|    8772|
+--------+



In [5]:
df = raw_data

tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
df = tokenizer.transform(df)
df.show(5)

Py4JJavaError: An error occurred while calling o53.showString.
: org.apache.spark.SparkException: Failed to execute user defined function(Tokenizer$$Lambda$2398/2140963301: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.Cast_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.writeFields_0_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.TakeOrderedAndProjectExec.$anonfun$executeCollect$2(limit.scala:190)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.map(Traversa

Sentiment analysis by wordlist

In [7]:
wl = WordList()
wl.analyze('this is an awesome test'.split(' '))
# creating an user-define-function to be applies to the tokens column
udf_sentiment_analysis = udf(lambda tokens:wl.analyze(tokens), IntegerType())

In [8]:
df = df.withColumn('sentiment', udf_sentiment_analysis(col('tokens')))

In [22]:
df.select(['text', 'sentiment']).show(10, truncate=False)

Py4JJavaError: An error occurred while calling o143.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)